In [1]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf
#Leemos el csv
evaluation = pandas.read_csv('turkiye-student-evaluation_R_Specific.csv', header=0)

In [2]:
#Mostramos las 10 primeras lineas para ver que se ha leido bien
evaluation.head(10)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
1,1,2,1,0,4,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,1,2,1,2,4,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,1,2,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,2,1,3,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,1,2,1,1,3,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
9,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,1,2,1,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [99]:
#Seleccionamos los atributos 
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos[5818,:])

[1.   0.   0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]


In [4]:
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
#scaler = preprocessing.MinMaxScaler()
#objetivo = scaler.fit_transform(objetivo.reshape(-1, 1))
print(objetivo)

[3. 3. 5. ... 5. 1. 1.]


In [5]:
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

In [6]:
normalizador = keras.layers.Normalization()
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()
red_evaluation.add(keras.layers.Input(shape=(31,)))
red_evaluation.add(normalizador)
red_evaluation.add(keras.layers.Dense(70, activation='relu'))
red_evaluation.add(keras.layers.Dense(70, activation='sigmoid'))
red_evaluation.add(keras.layers.Dense(1, activation='linear'))
#red_evaluation.add(keras.layers.Lambda(lambda x: x * 5))


In [7]:
red_evaluation.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 31)               63        
 n)                                                              
                                                                 
 dense (Dense)               (None, 70)                2240      
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 1)                 71        
                                                                 
Total params: 7,344
Trainable params: 7,281
Non-trainable params: 63
_________________________________________________________________


In [8]:

red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
20/20 [==============================] - 1s 2ms/step - loss: 1.7102 - accuracy: 0.1326 - mae: 0.9641  
Epoch 2/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4258 - accuracy: 0.1407 - mae: 0.4628
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.3175 - accuracy: 0.1407 - mae: 0.3642
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2912 - accuracy: 0.1407 - mae: 0.3351
Epoch 5/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2793 - accuracy: 0.1407 - mae: 0.3188
Epoch 6/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2721 - accuracy: 0.1407 - mae: 0.3079
Epoch 7/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2662 - accuracy: 0.1407 - mae: 0.3001
Epoch 8/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2611 - accuracy: 0.1407 - mae: 0.2949
Epoch 9/10
20/20 [==============================] - 0s 2ms/step - loss: 0.2573 - accuracy: 0.1407 - ma

In [9]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdObj = 1.0
x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
while x[0]>perdObj:
    red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)
    x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
    
print(x[0])

28/28 [==============================] - 0s 2ms/step - loss: 0.3061 - accuracy: 0.1329 - mae: 0.3105
0.3061012625694275


In [10]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

28/28 [==============================] - 0s 1ms/step - loss: 0.3061 - accuracy: 0.1329 - mae: 0.3105


[0.3061012625694275, 0.13287514448165894, 0.31046122312545776]

In [11]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=atributos[5809]
predicciones = red_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos[5810]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5811]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5812]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5813]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5814]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5815]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5816]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5817]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5818]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5819]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)

1/1 [==============================] - 0s 100ms/step
Esperado 4: [[3.9799373]]
1/1 [==============================] - 0s 24ms/step
Esperado 3: [[2.9219272]]
1/1 [==============================] - 0s 24ms/step
Esperado 3: [[3.0815432]]
1/1 [==============================] - 0s 24ms/step
Esperado 1: [[1.0223191]]
1/1 [==============================] - 0s 24ms/step
Esperado 3: [[2.8952112]]
1/1 [==============================] - 0s 15ms/step
Esperado 1: [[1.2187984]]
1/1 [==============================] - 0s 24ms/step
Esperado 1: [[1.0223191]]
1/1 [==============================] - 0s 25ms/step
Esperado 5: [[4.751582]]
1/1 [==============================] - 0s 24ms/step
Esperado 5: [[4.799588]]
1/1 [==============================] - 0s 24ms/step
Esperado 1: [[1.015504]]
1/1 [==============================] - 0s 24ms/step
Esperado 1: [[1.015504]]


In [12]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_evaluation = RandomForestRegressor()
forest_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento)
evaluaciones = forest_evaluation.predict(atributos_prueba)
mse = mean_squared_error(objetivo_prueba, evaluaciones)
print("Error cuadrático medio (MSE) del modelo:", mse)

Error cuadrático medio (MSE) del modelo: 0.24816278161822028


In [13]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
atributos2 = atributos[:, np.newaxis]
X=atributos2[5809]
predicciones = forest_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos2[5810]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5811]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5812]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5813]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5814]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5815]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5816]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5817]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5818]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5819]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)

Esperado 4: [4.]
Esperado 3: [3.]
Esperado 3: [3.]
Esperado 1: [1.]
Esperado 3: [3.]
Esperado 1: [1.19]
Esperado 1: [1.]
Esperado 5: [4.908]
Esperado 5: [5.]
Esperado 1: [1.]
Esperado 1: [1.]


In [14]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf

#Leemos el csv
#bikes = pandas.read_csv('hour.csv', header=0)
bikes = pandas.read_csv('hour2.csv',header = 0)


In [15]:
#Mostramos las 10 primeras lineas para ver que se ha leido bién
bikes.head(10)

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [16]:
#Seleccionamos los atributos y los nomalizamos
attr = bikes.loc[:, 'season':'registered']
attr = attr.to_numpy()
scaler = preprocessing.MinMaxScaler()
attr = scaler.fit_transform(attr)
print(attr)

[[0.         0.         0.         ... 0.         0.00817439 0.01467269]
 [0.         0.         0.         ... 0.         0.02179837 0.03611738]
 [0.         0.         0.         ... 0.         0.01362398 0.03047404]
 ...
 [0.         1.         1.         ... 0.19301751 0.01907357 0.09367946]
 [0.         1.         1.         ... 0.15786999 0.03542234 0.05417607]
 [0.         1.         1.         ... 0.15786999 0.03269755 0.04176072]]


In [17]:
#Seleccionamos los objetivos
obj = bikes['cnt']
obj = bikes['cnt'].to_numpy(dtype=np.float32)
#scaler = preprocessing.MinMaxScaler()
#objetivo = scaler.fit_transform(objetivo.reshape(-1, 1))
print(obj)

[16. 40. 32. ... 90. 61. 49.]


In [18]:
#Cogemos un 85% de los datos para entrenar y un 15% de los datos para evaluar el modelo
(attr_entrenamiento, attr_prueba,
 obj_entrenamiento, obj_prueba) = model_selection.train_test_split(
    attr, obj, test_size=.10)

In [19]:
#Creamos la red neuronal 
normalizador = keras.layers.Normalization()
normalizador.adapt(attr_entrenamiento)
red_bikes = keras.Sequential()
red_bikes.add(keras.layers.Input(shape=(14,))) #añadimos la capa de entrada con 14 neuronas
red_bikes.add(normalizador)#Aplicamos un normalizador
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(1, activation='linear'))#Necesitamos una capa de salida linear ya que los valores objetivos son lineares
#red_bikes.add(keras.layers.Lambda(lambda x: tf.cast(tf.round(x), dtype=tf.int32)))
#red_bikes.add(keras.layers.Lambda(lambda x: round(x)))


In [20]:
#Vemos un resumen de la red
red_bikes.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 14)               29        
 ion)                                                            
                                                                 
 dense_3 (Dense)             (None, 100)               1500      
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11,730
Trainable params: 11,701
Non-trainable params: 29
_________________________________________________________________


In [21]:
#Compilamos y entrenamos la red neuronal, buscamos minimizar la función de perdida
red_bikes.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
62/62 [==============================] - 1s 3ms/step - loss: 12604.8691 - accuracy: 0.0086 - mae: 79.1881
Epoch 2/10
62/62 [==============================] - 0s 3ms/step - loss: 2646.2527 - accuracy: 0.0086 - mae: 36.2257
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 1038.3522 - accuracy: 0.0088 - mae: 22.0724
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 951.6528 - accuracy: 0.0086 - mae: 21.7812
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 449.8022 - accuracy: 0.0087 - mae: 14.5733
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 540.2825 - accuracy: 0.0088 - mae: 18.9546
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 321.9869 - accuracy: 0.0088 - mae: 13.6415
Epoch 8/10
62/62 [==============================] - 0s 3ms/step - loss: 197.7039 - accuracy: 0.0088 - mae: 10.3707
Epoch 9/10
62/62 [==============================] - 0s 4ms/step - loss: 353.

In [22]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdidaObj = 1.0
x=red_bikes.evaluate(attr_prueba, obj_prueba)
while x[0]>perdidaObj:
    red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)
    x=red_bikes.evaluate(attr_prueba, obj_prueba)
    
print(x[0])

55/55 [==============================] - 0s 1ms/step - loss: 385.5063 - accuracy: 0.0115 - mae: 16.2184
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 272.1497 - accuracy: 0.0088 - mae: 12.9501
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 153.7853 - accuracy: 0.0088 - mae: 9.8920
Epoch 3/10
62/62 [==============================] - 0s 3ms/step - loss: 265.3095 - accuracy: 0.0084 - mae: 12.4868
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 181.5841 - accuracy: 0.0085 - mae: 10.2544
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 101.7851 - accuracy: 0.0088 - mae: 7.7663
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 60.1608 - accuracy: 0.0088 - mae: 5.8668
Epoch 7/10
62/62 [==============================] - 0s 3ms/step - loss: 170.0156 - accuracy: 0.0086 - mae: 10.3608
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 71.7742 - accuracy: 0.0

62/62 [==============================] - 0s 3ms/step - loss: 29.3736 - accuracy: 0.0088 - mae: 4.3357
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 12.2581 - accuracy: 0.0088 - mae: 2.5968
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 58.1585 - accuracy: 0.0088 - mae: 5.6827
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 34.5530 - accuracy: 0.0088 - mae: 4.2598
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 88.1723 - accuracy: 0.0115 - mae: 6.4067
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 16.8142 - accuracy: 0.0088 - mae: 3.1138
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 12.8507 - accuracy: 0.0088 - mae: 2.6940
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 8.9861 - accuracy: 0.0088 - mae: 2.3597
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 10.4760 - accuracy: 0.0088 - mae: 2

62/62 [==============================] - 0s 3ms/step - loss: 4.4928 - accuracy: 0.0088 - mae: 1.6110
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 4.2088 - accuracy: 0.0088 - mae: 1.6113
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 16.5503 - accuracy: 0.0088 - mae: 3.3554
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 11.3885 - accuracy: 0.0088 - mae: 2.7966
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 17.3381 - accuracy: 0.0088 - mae: 3.3673
Epoch 7/10
62/62 [==============================] - 0s 3ms/step - loss: 16.2054 - accuracy: 0.0088 - mae: 3.4356
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 8.7259 - accuracy: 0.0088 - mae: 2.3553
Epoch 9/10
62/62 [==============================] - 0s 4ms/step - loss: 18.9898 - accuracy: 0.0088 - mae: 3.4077
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 37.6791 - accuracy: 0.0115 - mae: 4.8

62/62 [==============================] - 0s 3ms/step - loss: 10.0019 - accuracy: 0.0088 - mae: 2.6292
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 18.1669 - accuracy: 0.0115 - mae: 3.0611
Epoch 1/10
62/62 [==============================] - 0s 4ms/step - loss: 9.8602 - accuracy: 0.0088 - mae: 2.6450
Epoch 2/10
62/62 [==============================] - 0s 4ms/step - loss: 10.1575 - accuracy: 0.0088 - mae: 2.7357
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 5.5682 - accuracy: 0.0088 - mae: 1.9113
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 6.3397 - accuracy: 0.0088 - mae: 2.0534
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 16.3664 - accuracy: 0.0088 - mae: 3.5688
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 7.9627 - accuracy: 0.0088 - mae: 2.3796
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 9.7065 - accuracy: 0.0088 - mae: 2.669

62/62 [==============================] - 0s 3ms/step - loss: 4.4676 - accuracy: 0.0086 - mae: 1.7121
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 3.1030 - accuracy: 0.0088 - mae: 1.3585
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 1.0930 - accuracy: 0.0088 - mae: 0.8483
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 1.6563 - accuracy: 0.0088 - mae: 1.0033
Epoch 9/10
62/62 [==============================] - 0s 4ms/step - loss: 14.8601 - accuracy: 0.0083 - mae: 3.4106
Epoch 10/10
55/55 [==============================] - 0s 970us/step - loss: 27.8676 - accuracy: 0.0104 - mae: 4.4685
Epoch 1/10
62/62 [==============================] - 0s 3ms/step - loss: 12.4272 - accuracy: 0.0088 - mae: 3.0225
Epoch 2/10
62/62 [==============================] - 0s 3ms/step - loss: 5.8247 - accuracy: 0.0088 - mae: 2.0554
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 7.2159 - accuracy: 0.0085 - mae: 2.30

...

62/62 [==============================] - 0s 2ms/step - loss: 2.3207 - accuracy: 0.0086 - mae: 1.2541
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 2.6689 - accuracy: 0.0086 - mae: 1.3690
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 9.4358 - accuracy: 0.0109 - mae: 2.5285
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 4.4071 - accuracy: 0.0086 - mae: 1.7481
Epoch 2/10
62/62 [==============================] - 0s 3ms/step - loss: 2.8545 - accuracy: 0.0088 - mae: 1.4156
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 9.1290 - accuracy: 0.0080 - mae: 2.6910
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 17.5667 - accuracy: 0.0082 - mae: 3.7682
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 5.2844 - accuracy: 0.0086 - mae: 1.9917
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 5.1706 - accuracy: 0.0084 - mae: 1.9443
E

62/62 [==============================] - 0s 4ms/step - loss: 4.5301 - accuracy: 0.0077 - mae: 1.7899
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 2.8479 - accuracy: 0.0082 - mae: 1.4322
Epoch 6/10
62/62 [==============================] - 0s 2ms/step - loss: 3.5021 - accuracy: 0.0086 - mae: 1.6235
Epoch 7/10
62/62 [==============================] - 0s 2ms/step - loss: 3.5620 - accuracy: 0.0081 - mae: 1.6355
Epoch 8/10
62/62 [==============================] - 0s 2ms/step - loss: 8.4713 - accuracy: 0.0080 - mae: 2.5770
Epoch 9/10
62/62 [==============================] - 0s 2ms/step - loss: 3.8966 - accuracy: 0.0088 - mae: 1.6179
Epoch 10/10
55/55 [==============================] - 0s 953us/step - loss: 9.9654 - accuracy: 0.0115 - mae: 2.5418
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 5.1168 - accuracy: 0.0084 - mae: 1.9819
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 7.2810 - accuracy: 0.0084 - mae: 2.3177


55/55 [==============================] - 0s 1ms/step - loss: 1.0193 - accuracy: 0.0115 - mae: 0.7849
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 1.5565 - accuracy: 0.0085 - mae: 0.9620
Epoch 2/10
62/62 [==============================] - 0s 2ms/step - loss: 2.7505 - accuracy: 0.0082 - mae: 1.4177
Epoch 3/10
62/62 [==============================] - 0s 2ms/step - loss: 7.1344 - accuracy: 0.0082 - mae: 2.2760
Epoch 4/10
62/62 [==============================] - 0s 2ms/step - loss: 5.4545 - accuracy: 0.0083 - mae: 2.0895
Epoch 5/10
62/62 [==============================] - 0s 3ms/step - loss: 4.0250 - accuracy: 0.0084 - mae: 1.7522
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 0.8521 - accuracy: 0.0088 - mae: 0.7302
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 0.6777 - accuracy: 0.0088 - mae: 0.6608
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 1.1939 - accuracy: 0.0088 - mae: 0.8675
Epo

62/62 [==============================] - 0s 3ms/step - loss: 7.4131 - accuracy: 0.0084 - mae: 2.3812
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 7.1049 - accuracy: 0.0083 - mae: 2.2600
Epoch 9/10
62/62 [==============================] - 0s 4ms/step - loss: 1.4881 - accuracy: 0.0086 - mae: 1.0090
Epoch 10/10
55/55 [==============================] - 0s 971us/step - loss: 4.5997 - accuracy: 0.0046 - mae: 1.6412
Epoch 1/10
62/62 [==============================] - 0s 2ms/step - loss: 1.8089 - accuracy: 0.0082 - mae: 1.0918
Epoch 2/10
62/62 [==============================] - 0s 4ms/step - loss: 0.8713 - accuracy: 0.0088 - mae: 0.7499
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 5.3231 - accuracy: 0.0079 - mae: 2.0329
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 4.7745 - accuracy: 0.0085 - mae: 1.8508
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 0.5753 - accuracy: 0.0088 - mae: 0.6072


62/62 [==============================] - 0s 4ms/step - loss: 9.6384 - accuracy: 0.0079 - mae: 2.7023
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 1.6034 - accuracy: 0.0086 - mae: 1.0127
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 1.1967 - accuracy: 0.0086 - mae: 0.8709
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 2.2099 - accuracy: 0.0083 - mae: 1.2627
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 6.7845 - accuracy: 0.0079 - mae: 2.3533
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 2.7277 - accuracy: 0.0082 - mae: 1.4190
Epoch 9/10
62/62 [==============================] - 0s 4ms/step - loss: 4.3721 - accuracy: 0.0075 - mae: 1.8348
Epoch 10/10
55/55 [==============================] - 0s 1ms/step - loss: 1.4614 - accuracy: 0.0115 - mae: 0.9293
Epoch 1/10
62/62 [==============================] - 0s 3ms/step - loss: 2.2952 - accuracy: 0.0082 - mae: 1.3019
Ep

In [23]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_bikes.evaluate(attr_prueba, obj_prueba)

55/55 [==============================] - 0s 2ms/step - loss: 0.7918 - accuracy: 0.0115 - mae: 0.6337


[0.7917532920837402, 0.011507479473948479, 0.6337389945983887]

In [24]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=attr[17368]
predicciones = red_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr[17369]
predicciones = red_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr[17370]
predicciones = red_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr[17371]
predicciones = red_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr[17372]
predicciones = red_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr[17373]
predicciones = red_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr[17374]
predicciones = red_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr[17375]
predicciones = red_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr[17376]
predicciones = red_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr[17377]
predicciones = red_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr[17378]
predicciones = red_bikes.predict(X)
print('Esperado 49:',predicciones)

1/1 [==============================] - 0s 112ms/step
Esperado 203: [[202.72833]]
1/1 [==============================] - 0s 32ms/step
Esperado 247: [[247.45126]]
1/1 [==============================] - 0s 33ms/step
Esperado 315: [[315.5245]]
1/1 [==============================] - 0s 24ms/step
Esperado 214: [[214.99004]]
1/1 [==============================] - 0s 16ms/step
Esperado 164: [[164.0082]]
1/1 [==============================] - 0s 19ms/step
Esperado 122: [[122.89195]]
1/1 [==============================] - 0s 25ms/step
Esperado 119: [[119.91462]]
1/1 [==============================] - 0s 17ms/step
Esperado 89: [[88.61764]]
1/1 [==============================] - 0s 20ms/step
Esperado 90: [[89.52659]]
1/1 [==============================] - 0s 18ms/step
Esperado 61: [[61.515926]]
1/1 [==============================] - 0s 17ms/step
Esperado 49: [[49.03857]]


In [25]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_bikes = RandomForestRegressor()
forest_bikes.fit(attr_entrenamiento, obj_entrenamiento)
evaluaciones = forest_bikes.predict(attr_prueba)
mse = mean_squared_error(obj_prueba, evaluaciones)
print("Error cuadrático medio (MSE) del modelo:", mse)

Error cuadrático medio (MSE) del modelo: 2.8569749136938993


In [26]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
attr2 = attr[:, np.newaxis]
X=attr2[17368]
predicciones = forest_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr2[17369]
predicciones = forest_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr2[17370]
predicciones = forest_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr2[17371]
predicciones = forest_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr2[17372]
predicciones = forest_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr2[17373]
predicciones = forest_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr2[17374]
predicciones = forest_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr2[17375]
predicciones = forest_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr2[17376]
predicciones = forest_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr2[17377]
predicciones = forest_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr2[17378]
predicciones = forest_bikes.predict(X)
print('Esperado 49:',predicciones)

Esperado 203: [202.76]
Esperado 247: [247.42]
Esperado 315: [314.52]
Esperado 214: [214.05]
Esperado 164: [164.02]
Esperado 122: [121.49]
Esperado 119: [118.88]
Esperado 89: [89.02]
Esperado 90: [90.]
Esperado 61: [60.96]
Esperado 49: [48.91]


In [27]:
#Metodo LIME
import random
from sklearn.linear_model import Ridge



"""
N es el n´umero de permutaciones a realizar
f es el modelo a explicar
X′ ← {} muestras perturbadas
R ← {} representaciones
W ← {} las distancias entre la muestra x y sus perturbaciones
for 1
to
N do
Selecciona k atributos aleatoriamente
x′ ← una perturbaci´on de x donde se perturban los k atributos anteriores.
w ← la distancia entre x y x′
r ← la representaci´on de x′ respecto a x
X′ ← X′ ∪ x′
R ← R ∪ r
W ← R ∪ w
end for
Y ′ ← f(X′) las predicciones de las perturbaciones
G ← modelo ridge entrenado con R para predecir Y ′ y ponderando cada
muestra con W
return los par´ametros de G
"""

def explain_model(f, x, N,M):
    #f es el modelo a explicar
    #N es el número de permutaciones a realizar
    Xi = [] #muestras perturbadas
    R = [] #representaciones
    W = [] #Las distancias entre la muestra x y sus pertubaciones
    for i in range(N):
        k = random.randint(1, len(x)) #Escojo un número aletario que representa el número de los atributos a seleccionar
        perturbed_x = x.copy() #copio la muestra original
        for j in range(k): 
            perturbed_attr = random.randint(0,len(x)-1)#random.choice(list(x.keys())) Voy escojiento los atributos que perturbare de forma aleatoria
            mx = abs(max([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor máximo
            mn = abs(min([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor mínimo
            perturbed_x[perturbed_attr] = random.uniform(mn, mx) # pertubo el atributo con un valor aleatorio ente mn y mx
        w = abs(sum([x[attr] - perturbed_x[attr] for attr in range(0,len(x)-1)])) #Calculo la distancia entre el original y el perturbado
        r = [0  if perturbed_x[attr] == x[attr] else 1 for attr in range(0,len(x)-1)] #miramos la representacion del pertubado respecto al original utilizando un operador ternario como solemos hacer en java
        Xi.append(perturbed_x) #Acumulo en la lista los perturbados
        R.append(r)#Acumulamos en la lista de representaciones
        W.append(w)#Añadimos las distancias a la lista
    Y_perturbed = f(Xi)#aplicamos el modelo f
    G = Ridge()
    G.fit(R, Y_perturbed, sample_weight=W)
    return G.get_params() #Devolvemos los parametros


In [28]:
explain_model(red_evaluation,atributos[5809],1,atributos)

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'normalize': 'deprecated',
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.001}

In [29]:
def explain_model(f, X, N, M):
    # f es el modelo a explicar
    # X es una lista de ejemplos a los que se aplicará LIME
    # N es el número de permutaciones a realizar
    # M es una matriz de ejemplos para calcular mx y mn

    explanations = []  # Lista para almacenar las explicaciones de cada ejemplo

    for x in X:
        Xi = []  # muestras perturbadas
        R = []  # representaciones
        W = []  # Las distancias entre el ejemplo x y sus perturbaciones

        for i in range(N):
            k = random.randint(1, len(x))  # Escojo un número aleatorio que representa el número de atributos a seleccionar
            perturbed_x = x.copy()  # Copio el ejemplo original

            for j in range(k):
                perturbed_attr = random.randint(0, len(x)-1)  # Voy seleccionando los atributos que perturbaré de forma aleatoria
                mx = abs(max([a[perturbed_attr] for a in M]))  # Calculo el valor máximo para el atributo seleccionado
                mn = abs(min([a[perturbed_attr] for a in M]))  # Calculo el valor mínimo para el atributo seleccionado
                perturbed_x[perturbed_attr] = random.uniform(mn, mx)  # Perturbo el atributo con un valor aleatorio entre mn y mx

            w = abs(sum([x[attr] - perturbed_x[attr] for attr in range(len(x))]))  # Calculo la distancia entre el original y el perturbado
            r = [0 if perturbed_x[attr] == x[attr] else 1 for attr in range(len(x))]  # Calculo la representación del perturbado respecto al original utilizando un operador ternario

            Xi.append(perturbed_x)  # Acumulo en la lista los ejemplos perturbados
            R.append(r)  # Acumulo en la lista de representaciones
            W.append(w)  # Añado las distancias a la lista

        Y_perturbed = f(Xi)  # Aplico el modelo f a los ejemplos perturbados
        G = Ridge()
        G.fit(R, Y_perturbed, sample_weight=W)
        explanations.append(G.get_params())  # Almaceno los parámetros del modelo G como la explicación para el ejemplo x

    return explanations

In [41]:
atributivos = atributos[5800:5809, :]

explain_model(red_evaluation,atributivos,1,atributos)

[{'alpha': 1.0,
  'copy_X': True,
  'fit_intercept': True,
  'max_iter': None,
  'normalize': 'deprecated',
  'positive': False,
  'random_state': None,
  'solver': 'auto',
  'tol': 0.001},
 {'alpha': 1.0,
  'copy_X': True,
  'fit_intercept': True,
  'max_iter': None,
  'normalize': 'deprecated',
  'positive': False,
  'random_state': None,
  'solver': 'auto',
  'tol': 0.001},
 {'alpha': 1.0,
  'copy_X': True,
  'fit_intercept': True,
  'max_iter': None,
  'normalize': 'deprecated',
  'positive': False,
  'random_state': None,
  'solver': 'auto',
  'tol': 0.001},
 {'alpha': 1.0,
  'copy_X': True,
  'fit_intercept': True,
  'max_iter': None,
  'normalize': 'deprecated',
  'positive': False,
  'random_state': None,
  'solver': 'auto',
  'tol': 0.001},
 {'alpha': 1.0,
  'copy_X': True,
  'fit_intercept': True,
  'max_iter': None,
  'normalize': 'deprecated',
  'positive': False,
  'random_state': None,
  'solver': 'auto',
  'tol': 0.001},
 {'alpha': 1.0,
  'copy_X': True,
  'fit_intercept

In [106]:
import numpy as np

def identidad(X, epsilon):
    """
    Calcula la métrica de Identidad para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        epsilon (float): Valor de perturbación.

    Returns:
        bool: True si se cumple la métrica de Identidad para todas las parejas de instancias, False en caso contrario.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    # Calcula las distancias entre todas las parejas de instancias
    distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)

    # Verifica si se cumple la métrica de Identidad para todas las parejas de instancias
    for i in range(n):
        for j in range(i + 1, n):
            if distances[i, j] == 0 and np.linalg.norm(epsilon * X[i] - epsilon * X[j]) != 0:
                return False

    return True


In [107]:
explain = explain_model(red_evaluation,atributivos,1,atributos)
identidad(atributos[5814:5815],[4,3])

True

In [47]:
#METODO IDENTIDAD
import pandas
import numpy as np
from scipy.spatial.distance import cdist
from scipy.stats import pearsonr
from scipy.special import lambertw
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Creamos dos instancias de datos y sus respectivas explicaciones LIME
xa = np.array([0.5, 0.7, 0.1, 0.3])
xb = np.array([0.3, 0.9, 0.2, 0.1])
ea = np.array([0.2, 0.3, 0.1, 0.4])
eb = np.array([0.3, 0.4, 0.5, 0.1])

# Calculamos la distancia entre las instancias de datos
data_distance = cdist(xa.reshape(1,-1), xb.reshape(1,-1))

# Calculamos la distancia entre las explicaciones LIME
exp_distance = cdist(ea.reshape(1,-1), eb.reshape(1,-1))

# Verificamos si la métrica Identidad se cumple
if data_distance == 0 and exp_distance != 0:
    print("La métrica Identidad no se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Identidad se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")


La métrica Identidad no se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.


In [108]:
import numpy as np

def separabilidad(X, epsilon):
    """
    Calcula la métrica de Separabilidad para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        epsilon (float): Valor de perturbación.

    Returns:
        bool: True si se cumple la métrica de Separabilidad para todas las parejas de instancias, False en caso contrario.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    # Calcula las distancias entre todas las parejas de instancias
    distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)

    # Verifica si se cumple la métrica de Separabilidad para todas las parejas de instancias
    for i in range(n):
        for j in range(i + 1, n):
            if distances[i, j] != 0 and np.linalg.norm(epsilon * X[i] - epsilon * X[j]) <= 0:
                return False

    return True


In [34]:
#METODO SEPARABILIDAD
# Creamos dos instancias de datos y sus respectivas explicaciones LIME
xa = np.array([0.5, 0.7, 0.1, 0.3])
xb = np.array([0.3, 0.9, 0.2, 0.1])
ea = np.array([0.2, 0.3, 0.1, 0.4])
eb = np.array([0.3, 0.4, 0.5, 0.1])

# Calculamos la distancia entre las instancias de datos
data_distance = cdist(xa.reshape(1,-1), xb.reshape(1,-1))

# Calculamos la distancia entre las explicaciones LIME
exp_distance = cdist(ea.reshape(1,-1), eb.reshape(1,-1))

# Verificamos si la métrica Separabilidad se cumple
if data_distance != 0 and exp_distance <= 0:
    print("La métrica Separabilidad no se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Separabilidad se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.")


La métrica Separabilidad se cumple para este par de instancias de datos y sus explicaciones LIME correspondientes.


In [ ]:
import numpy as np

def estabilidad(X, epsilon):
    """
    Calcula la métrica de Estabilidad para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        epsilon (float): Valor de perturbación.

    Returns:
        bool: True si se cumple la métrica de Estabilidad para todas las características, False en caso contrario.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    # Calcula las distancias entre todas las parejas de instancias
    distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)

    # Calcula las distancias entre las instancias perturbadas
    perturbed_distances = np.linalg.norm(epsilon * X[:, np.newaxis] - epsilon * X, axis=2)

    # Verifica si se cumple la métrica de Estabilidad para todas las características
    for i in range(m):
        max_distance = np.max(distances[:, i])
        min_perturbed_distance = np.min(perturbed_distances[:, i])

        if min_perturbed_distance <= max_distance:
            return False

    return True


In [35]:
#METODO ESTABILIDAD
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Creamos las distancias entre las instancias de datos y sus respectivas explicaciones LIME
data_distances = cdist(x, x[0].reshape(1,-1))
exp_distances = cdist(ex, ex[0].reshape(1,-1))

# Calculamos el coeficiente de correlación de Pearson entre las distancias
correlation, _ = pearsonr(data_distances.flatten(), exp_distances.flatten())

# Verificamos si la métrica Estabilidad se cumple
if correlation > 0:
    print("La métrica Estabilidad se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.")
else:
    print("La métrica Estabilidad no se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.")


La métrica Estabilidad se cumple para este conjunto de instancias de datos y sus explicaciones LIME correspondientes.


In [109]:
import numpy as np

def coherencia(X, epsilon):
    """
    Calcula la métrica de Coherencia para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        epsilon (float): Valor de perturbación.

    Returns:
        float: Valor de la métrica de Coherencia.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    alpha_values = np.zeros(m)  # arreglo para almacenar los valores de alpha

    # Calcula las distancias entre todas las parejas de instancias
    distances = np.linalg.norm(X[:, np.newaxis] - X, axis=2)

    # Calcula las distancias entre las instancias perturbadas
    perturbed_distances = np.linalg.norm(epsilon * X[:, np.newaxis] - epsilon * X, axis=2)

    # Calcula los valores de alpha para cada característica
    for i in range(m):
        differences = np.exp(-distances[:, i]) - np.exp(-perturbed_distances[:, i])
        alpha_values[i] = np.mean(differences) / np.std(differences)

    return alpha_values


In [36]:
#METODO COHERENCIA
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=1))
exp_weights = np.exp(-np.linalg.norm(ex - ex.mean(axis=0), axis=1))

# Calculamos los logaritmos naturales de los pesos y los sumamos
log_sum_data = np.sum(np.log(data_weights))
log_sum_exp = np.sum(np.log(exp_weights))

# Calculamos la métrica Coherencia
alpha = np.exp(log_sum_data - log_sum_exp + lambertw(0.0, -np.exp(log_sum_data - log_sum_exp)).real)
alpha /= len(x)

# Imprimimos el valor de la métrica Coherencia
print("El valor de la métrica Coherencia es:", alpha)


TypeError: ufunc '_lambertw' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [ ]:
import numpy as np

def completitud(alpha_values):
    """
    Calcula la métrica de Completitud para el método LIME.

    Args:
        alpha_values (ndarray): Arreglo de valores de alpha calculados para cada característica.

    Returns:
        float: Valor de la métrica de Completitud.
    """

    alpha_exp = np.exp(alpha_values)  # Calcula la exponencial de los valores de alpha
    completeness = np.sum(alpha_exp) / (np.sum(alpha_exp) + 1)

    return completeness


In [ ]:
#METODO COMPLETITUD
# Creamos las instancias de datos y sus respectivas explicaciones LIME
x = np.array([[0.5, 0.7, 0.1, 0.3], [0.3, 0.9, 0.2, 0.1], [0.1, 0.2, 0.8, 0.6]])
ex = np.array([[0.2, 0.3, 0.1, 0.4], [0.3, 0.4, 0.5, 0.1], [0.1, 0.3, 0.6, 0.2]])

# Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=1))
exp_weights = np.exp(-np.linalg.norm(ex - ex.mean(axis=0), axis=1))

# Calculamos los valores de ei y pi
ei = np.sum(exp_weights)
pi = np.sum(data_weights)

# Calculamos la métrica Completitud
gamma = ei / pi

# Imprimimos el valor de la métrica Completitud
print("El valor de la métrica Completitud es:", gamma)


In [ ]:
import numpy as np

def congruencia(alpha_values):
    """
    Calcula la métrica de Congruencia para el método LIME.

    Args:
        alpha_values (ndarray): Arreglo de valores de alpha calculados para cada característica.

    Returns:
        float: Valor de la métrica de Congruencia.
    """

    alpha_avg = np.mean(alpha_values)  # Coherencia promedio
    n = alpha_values.shape[0]  # Número de características

    # Calcula la métrica de Congruencia
    delta = np.sum(np.square(alpha_values - alpha_avg)) / n

    return delta


In [ ]:
#METODO CONGRUENCIA
# Creamos una lista de valores de coherencia para un conjunto de N muestras
alpha_values = [0.8, 0.6, 0.7, 0.9, 0.5]

# Calculamos el valor promedio de la coherencia
alpha = np.mean(alpha_values)

# Creamos una lista de valores de alpha_i para cada instancia de datos
alpha_i_values = [0.9, 0.7, 0.6, 0.8, 0.4]

# Calculamos el número de instancias de datos
N = len(alpha_i_values)

# Calculamos la suma de las diferencias cuadráticas entre alpha_i y alpha
diff_squares_sum = np.sum((np.array(alpha_i_values) - alpha) ** 2)

# Calculamos la métrica Congruencia
delta = np.sqrt((diff_squares_sum / N) / alpha)

# Imprimimos el valor de la métrica Congruencia
print("El valor de la métrica Congruencia es:", delta)


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

def selectividad(X, y, model):
    """
    Calcula la métrica de Selectividad para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        y (ndarray): Vector de etiquetas de forma (n,), donde n es el número de instancias.
        model: Modelo de aprendizaje automático ajustado sobre los datos completos.

    Returns:
        float: Área bajo la curva (AUC) que mide la selectividad al eliminar características una por una.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    auc_scores = []  # lista para almacenar los resultados del AUC

    # Realiza las predicciones sobre los datos completos
    y_pred_base = model.predict_proba(X)[:, 1]

    # Calcula el AUC para el modelo sobre los datos completos
    auc_base = roc_auc_score(y, y_pred_base)

    # Itera sobre cada característica y calcula el AUC al eliminarla
    for i in range(m):
        X_copy = X.copy()
        X_copy[:, i] = 0  # Establece en cero la i-ésima característica

        # Realiza las predicciones con la característica eliminada
        y_pred = model.predict_proba(X_copy)[:, 1]

        # Calcula el AUC y lo agrega a la lista de resultados
        auc_score = roc_auc_score(y, y_pred)
        auc_scores.append(auc_score)

    # Calcula el AUC promedio para la selectividad
    auc_selectivity = np.mean(auc_scores)

    return auc_selectivity


In [ ]:
#METODO SELECTIVIDAD
# Cargamos el dataset de cáncer de mama de scikit-learn
data = load_breast_cancer()

# Definimos las características y la variable objetivo
X = data['data']
y = data['target']

# Creamos un modelo de regresión logística
model = LogisticRegression()

# Entrenamos el modelo
model.fit(X, y)

# Obtenemos las predicciones del modelo en el conjunto de datos completo
y_pred = model.predict_proba(X)[:, 1]

# Calculamos el AUC para el modelo entrenado en el conjunto de datos completo
auc_full = roc_auc_score(y, y_pred)

# Ordenamos las características de más a menos relevantes según el modelo
idx_sorted = np.argsort(np.abs(model.coef_))[0][::-1]

# Creamos una lista para almacenar los AUC para cada característica eliminada
auc_scores = []

# Iteramos sobre las características de más a menos relevantes según el modelo
for i in range(X.shape[1]):
    # Establecemos la característica i en cero
    X_test = X.copy()
    X_test[:, idx_sorted[i]] = 0
    
    # Obtenemos las predicciones del modelo en el conjunto de datos con la característica i eliminada
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    # Calculamos el AUC para el modelo entrenado en el conjunto de datos con la característica i eliminada
    auc_test = roc_auc_score(y, y_pred_test)
    
    # Almacenamos el AUC para la característica i eliminada en la lista de AUC
    auc_scores.append(auc_test)

# Calculamos la métrica Selectividad como el área bajo la curva ROC para el gráfico de AUC vs características eliminadas
selectivity = np.trapz(auc_scores, dx=1)

# Imprimimos el valor de la métrica Selectividad
print("El valor de la métrica Selectividad es:", selectivity)
